# Convolutional Neural Network

In [1]:
import time
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

### Settings and Dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Hyperparameters
random_seed = 1
learning_rate = 0.05
num_epochs = 10
batch_size = 128
num_classes = 10


train_dataset = datasets.MNIST(root='data',  
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)


# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

### Model

In [3]:
class ConvNet(torch.nn.Module):

    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
    
        self.conv_1 = torch.nn.Conv2d(in_channels=1,
                                      out_channels=8,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1) 
        
        self.pool_1 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         padding=0)                                      
        
        self.conv_2 = torch.nn.Conv2d(in_channels=8,
                                      out_channels=16,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1)                 
                                   
        self.pool_2 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         padding=0) 

        self.linear_1 = torch.nn.Linear(7*7*16, num_classes)


    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(out)
        out = self.pool_1(out)

        out = self.conv_2(out)
        out = F.relu(out)
        out = self.pool_2(out)
        
        logits = self.linear_1(out.view(-1, 7*7*16))
        probas = F.softmax(logits, dim=1)
        return logits, probas

    
torch.manual_seed(random_seed)
model = ConvNet(num_classes=num_classes)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Train

In [4]:
def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    for features, targets in data_loader:
        features = features.to(device)
        targets = targets.to(device)
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
    

start_time = time.time()    
for epoch in range(num_epochs):
    model = model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)

        # FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        cost.backward()
        
        # UPDATE MODEL PARAMETERS
        optimizer.step()
        
        # LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))
    
    model = model.eval()
    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))

Epoch: 001/010 | Batch 000/469 | Cost: 2.3190
Epoch: 001/010 | Batch 050/469 | Cost: 1.8922
Epoch: 001/010 | Batch 100/469 | Cost: 0.7884
Epoch: 001/010 | Batch 150/469 | Cost: 0.4940
Epoch: 001/010 | Batch 200/469 | Cost: 0.3606
Epoch: 001/010 | Batch 250/469 | Cost: 0.2951
Epoch: 001/010 | Batch 300/469 | Cost: 0.2658
Epoch: 001/010 | Batch 350/469 | Cost: 0.4450
Epoch: 001/010 | Batch 400/469 | Cost: 0.1797
Epoch: 001/010 | Batch 450/469 | Cost: 0.1551
Epoch: 001/010 training accuracy: 93.29%
Epoch: 002/010 | Batch 000/469 | Cost: 0.2622
Epoch: 002/010 | Batch 050/469 | Cost: 0.2215
Epoch: 002/010 | Batch 100/469 | Cost: 0.1826
Epoch: 002/010 | Batch 150/469 | Cost: 0.1789
Epoch: 002/010 | Batch 200/469 | Cost: 0.2263
Epoch: 002/010 | Batch 250/469 | Cost: 0.0823
Epoch: 002/010 | Batch 300/469 | Cost: 0.0939
Epoch: 002/010 | Batch 350/469 | Cost: 0.2481
Epoch: 002/010 | Batch 400/469 | Cost: 0.0653
Epoch: 002/010 | Batch 450/469 | Cost: 0.2275
Epoch: 002/010 training accuracy: 95.34

### Test

In [5]:
with torch.set_grad_enabled(False):
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 98.39%
